Implementation Plan. Part 2 - for F-Schore values
1. Things to crawl to get f-score
   - ROA --> independent
   - CFO : cash flow from operations 
   - delta ROA 
   - Accrual
   - delta lever 
   - delta liquid 
   - equity offer
   - delta margin
   - delta turnover 

In [1]:
# Importing packages 
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import numpy as np
import pandas as pd

In [2]:
# Headless Chrome setting : v60 above, no physical browser
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [3]:
# Starting a browser(chrome) using selenium : Chrome requires a physical driver 
driver = webdriver.Chrome('/Users/daesikkim/Downloads/chromedriver', chrome_options=options)
driver.implicitly_wait(3)

In [4]:
# open a webpage
driver.get('http://finance.naver.com/item/coinfo.nhn?code=005930')

# move to a right frame
frame = driver.find_element_by_css_selector("#coinfo_cp")
driver.switch_to.frame(frame)

# click a right tab or button 
finstate_button = driver.find_element_by_css_selector("#header-menu > div.wrapper-menu > dl > dt:nth-of-type(3) > a")
finstate_button.click()

In [5]:
# get the page source - "Income Statement"
html_income = driver.page_source

In [6]:
# make the soup!
soup_income = BeautifulSoup(html_income, 'lxml')

Values required for ROA, CFO(cash flow from operations), delta ROA
1. for profitability 
   - net income / net income without extraordinary items : net income - extraordinary items(기타영업손익 및 기타영업외손익) 
   - total assets 
   - cash flow from operations 

In [7]:
# get the page source - "Balance sheet" 
balancesheet_button = driver.find_element_by_css_selector("#rpt_tab2")
balancesheet_button.click()
driver.implicitly_wait(3)

In [8]:
html_balance = driver.page_source

In [9]:
html_balance

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html xmlns="http://www.w3.org/1999/xhtml" lang="ko" style="" class=" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths"><head>\n\t<title>온라인기업정보 - 기업모니터 - 재무상태표</title>\n\t<link rel="shortcut icon" href="/favicon.ico" />\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\t<script async="" src="https://www.google-analytics.com/analytics.js"></script><script language="javascript" src="/include/domain.js" type="text/javascript"></script>\n\t<!--[if (!IE) | (gt IE 8)]>\n 

In [10]:
# make the soup! 
soup_balance = BeautifulSoup(html_balance, "lxml")

In [11]:
soup_balance

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html class=" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths" lang="ko" style="" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>온라인기업정보 - 기업모니터 - 재무상태표</title>
<link href="/favicon.ico" rel="shortcut icon"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script async="" src="https://www.google-analytics.com/analytics.js"></script><script language="javascript" src="/include/domain.js" type="text/javascript"></script>
<!--[if (!IE) | (gt IE 8)]>
    	<meta name="vi

In [12]:
# get the page source - "Cash Flow"
cashflow_button = driver.find_element_by_css_selector("#rpt_tab3")
cashflow_button.click()
driver.implicitly_wait(3)

In [13]:
html_cash = driver.page_source

In [14]:
html_cash

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html xmlns="http://www.w3.org/1999/xhtml" lang="ko" style="" class=" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths"><head>\n\t<title>온라인기업정보 - 기업모니터 - 현금흐름표</title>\n\t<link rel="shortcut icon" href="/favicon.ico" />\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n\t<script async="" src="https://www.google-analytics.com/analytics.js"></script><script language="javascript" src="/include/domain.js" type="text/javascript"></script>\n\t<!--[if (!IE) | (gt IE 8)]>\n 

In [15]:
# make the soup! 
soup_cash = BeautifulSoup(html_cash, "lxml")

Items required!!! 
1. From "Income Statement" 
   - net income / net income before extraordinary items --> 영업이익으로 대체 or 당기순이익 - 영업외이익
   - sales --> 매출액
   - profit --> 매출총이익(gross profit) 
   - COGS --> cost of goods sold(매출원가) 
   (gross margin --> sales - COGS)
2. From "Balance Sheet" 
   - total assets --> 자산총계 
   - long term debt --> 비유동부채(long term liabilities) or 장기차입금.. 
   - current asset --> 유동자산 
   - current liabilities --> 유동부채
   - Shareholder's equity --> 자본총계
3. From "Cash Flow" 
   - cash flow from operation --> 영업활동으로 인한 현금흐름

In [16]:
# values from 'Income Statement' 
operating_profit = soup_income.select('#table-content > table > tbody > tr:nth-of-type(59) > td.num')
extra_income = soup_income.select('#table-content > table > tbody > tr:nth-of-type(144) > td.num')
total_sales = soup_income.select('#table-content > table > tbody > tr:nth-of-type(1) > td.num')
gross_profit = soup_income.select('#table-content > table > tbody > tr:nth-of-type(26) > td.num')
cogs = soup_income.select('#table-content > table > tbody > tr:nth-of-type(15) > td.num')

In [17]:
# get only numbers >> use 'string' method 
yrly_op_profits = []
for i in operating_profit[0:5]: 
    i = i.string
    yrly_op_profits.append(float(i.replace(',', '')))

yrly_ext_incomes = []
for i in extra_income[0:5]: 
    i = i.string
    yrly_ext_incomes.append(float(i.replace(',', '')))

yrly_tot_sales = []
for i in total_sales[0:5]: 
    i = i.string
    yrly_tot_sales.append(float(i.replace(',', '')))

yrly_gros_profits = []
for i in gross_profit[0:5]: 
    i = i.string
    yrly_gros_profits.append(float(i.replace(',', '')))
    
yrly_cogs = [] 
for i in cogs[0:5]:
    i = i.string
    yrly_cogs.append(float(i.replace(',', '')))

In [18]:
# values from 'Balance Sheet' 
total_asset = soup_balance.select('#table-content > table > tbody > tr:nth-of-type(1) > td.num')
lt_debt = soup_balance.select('#table-content > table > tbody > tr:nth-of-type(152) > td.num')
lt_borrowing = soup_balance.select('#table-content > table > tbody > tr:nth-of-type(158) > td.num')
current_asset = soup_balance.select('#table-content > table > tbody > tr:nth-of-type(2) > td.num')
current_liabilities = soup_balance.select('#table-content > table > tbody > tr:nth-of-type(110) > td.num')
shareholder_equity = soup_balance.select('#table-content > table > tbody > tr:nth-of-type(188) > td.num')

In [19]:
# get only numbers >> use 'string' method 
yrly_tot_assets = [] 
yrly_lt_debts = [] 
yrly_lt_borrowings = [] 
yrly_current_assets = []
yrly_current_liabilities = []
yrly_shareholder_equities = []

for i in total_asset[0:5]:
    i = i.string
    yrly_tot_assets.append(float(i.replace(',', '')))
    
for i in lt_debt[0:5]:
    i = i.string
    yrly_lt_debts.append(float(i.replace(',', '')))
    
for i in lt_borrowing[0:5]:
    i = i.string
    yrly_lt_borrowings.append(float(i.replace(',', '')))

for i in current_asset[0:5]:
    i = i.string
    yrly_current_assets.append(float(i.replace(',', '')))
    
for i in current_liabilities[0:5]: 
    i = i.string
    yrly_current_liabilities.append(float(i.replace(',', '')))
    
for i in shareholder_equity[0:5]: 
    i = i.string
    yrly_shareholder_equities.append(float(i.replace(',', '')))

In [20]:
#values from 'Cash Flow'
cf_operation = soup_cash.select('#table-content > table > tbody > tr:nth-of-type(1) > td.num')

In [21]:
# get only numbers 
yrly_cf_operations = []
for i in cf_operation[0:5]: 
    i = i.string
    yrly_cf_operations.append(float(i.replace(',', '')))

In [22]:
# slicing and insert to DF 
year =[2012, 2013, 2014, 2015, 2016]

fscore_data = {'year' : year,
               'operating_profit' : yrly_op_profits,
               'extra_income' : yrly_ext_incomes, 
               'total_sales' : yrly_tot_sales, 
               'gross_profit' : yrly_gros_profits, 
               'cogs' : yrly_cogs, 
               'total_asset' : yrly_tot_assets,
               'lt_debt' : yrly_lt_debts, 
               'lt_borrowing' : yrly_lt_borrowings,
               'current_asset' : yrly_current_assets,
               'current_liabilities' : yrly_current_liabilities, 
               'shareholder_equity' : yrly_shareholder_equities,
               'cash_flow_operations' : yrly_cf_operations,}

In [23]:
fscore_data

{'cash_flow_operations': [379728.1, 467074.4, 369753.9, 400617.6, 473856.4],
 'cogs': [1266519.3, 1376963.1, 1282788.0, 1234821.2, 1202777.2],
 'current_asset': [872690.2, 1107602.7, 1151460.3, 1248147.3, 1414297.0],
 'current_liabilities': [469330.5, 513154.1, 520139.1, 505029.1, 547041.0],
 'extra_income': [-230.4, 8155.0, 15416.2, -20374.9, 7744.5],
 'gross_profit': [744516.8, 909963.6, 779271.9, 771713.6, 815890.3],
 'lt_borrowing': [35212.6, 9027.2, 239.9, 1936.0, 11791.1],
 'lt_debt': [126583.1, 127436.0, 103208.6, 126168.1, 145072.0],
 'operating_profit': [290493.4, 367850.1, 250250.7, 264134.4, 292406.7],
 'shareholder_equity': [1214802.1, 1500160.1, 1680881.9, 1790598.1, 1929630.3],
 'total_asset': [1810715.7, 2140750.2, 2304229.6, 2421795.2, 2621743.2],
 'total_sales': [2011036.1, 2286926.7, 2062059.9, 2006534.8, 2018667.5],
 'year': [2012, 2013, 2014, 2015, 2016]}

In [24]:
# creating DF 
df_fscore_data = pd.DataFrame(fscore_data, columns = ['id', 
                                                      'name', 
                                                      'year', 
                                                      'quarter', 
                                                      'operating_profit', 
                                                      'extra_income', 
                                                      'total_sales', 
                                                      'gross_profit', 
                                                      'cogs',
                                                      'total_asset', 
                                                      'lt_debt', 
                                                      'lt_borrowing', 
                                                      'current_asset', 
                                                      'current_liabilities', 
                                                      'shareholder_equity', 
                                                      'cash_flow_operations',])

In [25]:
df_fscore_data

,id,name,year,quarter,operating_profit,extra_income,total_sales,gross_profit,cogs,total_asset,lt_debt,lt_borrowing,current_asset,current_liabilities,shareholder_equity,cash_flow_operations
0,NaN,NaN,2012,NaN,290493.4,-230.4,2011036.1,744516.8,1266519.3,1810715.7,126583.1,35212.6,872690.2,469330.5,1214802.1,379728.1
1,NaN,NaN,2013,NaN,367850.1,8155.0,2286926.7,909963.6,1376963.1,2140750.2,127436.0,9027.2,1107602.7,513154.1,1500160.1,467074.4
2,NaN,NaN,2014,NaN,250250.7,15416.2,2062059.9,779271.9,1282788.0,2304229.6,103208.6,239.9,1151460.3,520139.1,1680881.9,369753.9
3,NaN,NaN,2015,NaN,264134.4,-20374.9,2006534.8,771713.6,1234821.2,2421795.2,126168.1,1936.0,1248147.3,505029.1,1790598.1,400617.6
4,NaN,NaN,2016,NaN,292406.7,7744.5,2018667.5,815890.3,1202777.2,2621743.2,145072.0,11791.1,1414297.0,547041.0,1929630.3,473856.4
